# La pile \[ *stack* \] système et les sous-routines \[ *subroutines* \]

Vous avez probablement déjà rencontré l'idée des données organisées en pile (sinon, voir l'encart ci-dessous) et vous avez peut-être déjà implémenté une pile dans un langage haut niveau, en utilisant un tableau et un pointeur. Vous avez peut être aussi appris que, sur votre ordinateur, il y a quelquechose qu'on appelle «pile système» - peut être simplement par accident - lorsqu'un programme produit une erreur du genre « *Stack Overflow Error* » (débordement de la pile). 

La pile système est le plus souvent réalisée directement au niveau du processeur. L'ARMlite dispose d'une telle pile ainsi que d'instructions spécifiques pour l'utiliser: `PUSH` (empiler) et `POP` (dépiler).

### Encart sur les piles

à faire ...

Il est souvent utile d'utiliser la **pile système** dans vos programmes en assembleur. Le programme qui suit demande à l'utilisateur de saisir une chaîne de caractère. Celle-ci est rangée à une certaine adresse mémoire. Ensuite, elle est inversée en plaçant les caractères un à un dans la pile (en utilisant l'adressage indexé) puis en les retirant un à un de cette même pile tout en les rangeant à des adresses mémoires successives. Enfin, on affiche la chaîne obtenue dans la console. Il y a certainement d'autre façon de réaliser cela, mais l'utilisation de la pile est simple et c'est un *motif de conception* très commun:

```
// Rôle des registres
    // R0: utilisé pour écrire à l'écran
    // R1: index de la chaine en mémoire
    // R2: contient chaque caractère de la chaîne
    MOV R0, #maChaine
    STR R0, .ReadString
    MOV R1, #0
    MOV R2, #0
    PUSH {R2} //de façon à détecter la fin de la chaîne lorsqu'on dépile les caractères

boucle1:
    LDRB R2, [R0+R1]
    CMP R2, #0
    BEQ toutDepiler
    PUSH {R2}
    ADD R1, R1, #1 //car nous travaillons un octet à la fois
    B boucle1

toutDepiler:
    MOV R1, #0

boucle2:
    POP {R2}
    CMP R2, #0
    BEQ ecrire
    STRB R2, [R0+R1]
    ADD R1, R1, #1
    B boucle2

ecrire:
    STR R0, .WriteString
    HALT

maChaine:
```

*Notes*: 
- `STRB` et `LDRB` (B pour *byte* - octet) fonctionnent de manière similaire à `STR` et `LDR` à cela près que leur deuxième opérande est l'adresse d'un octet (et non d'un mot).
- L'opérande de `PUSH` et `POP` est entouré par des accolades \[ *braces* \] - `{..}`. Cela est fait pour marquer qu'il est possible d'empiler ou de dépiler plusieurs registres en une seule instruction, ainsi les accolades définissent une liste de registres. Nous verrons comment un peu plus tard. 

#### Exercice 30

Faite tourner ce programme et faites une copie d'écran montrant la chaîne saisie et le résultat.

Ensuite, faites le tourner à nouveau en mode lent et, après avoir insérer la chaîne, observer l'évolution du registre **SP** (*Stack Pointer* - pointeur de pile). Décrire vos observations à ce sujet.

À présent, cliquer sur **Edit** puis sur **Submit** (juste pour réinitialiser la mémoire).

Modifier la valeur de page mémoire de 000 à ffe

Faites alors à nouveau tourner le programme en alternant entre le mode pas à pas et le mode lent si vous le souhaitez, en insérant cette fois la chaîne ABCDE (pour les reconnaître facilement sous forme hexadécimal pour leur code ASCII - de 0x41 à 0x45). Une fois que vous l'avez saisie, décrire ce qui se passe dans la page mémoire visible. Décrivez aussi comment cela semble relié à la valeur du registre `SP`.
____

On peut se demander pourquoi les valeurs (un caractère par mot dans ce cas) ne sont pas effacées lorsque la pile est vidée? Tout simplement parce qu'il n'y aucune nécessité de le faire - ce serait une perte de temps pour le processeur. Le pointeur de pile informe le processeur sur la position où empiler ou dépiler la prochaîne valeur (qui écrasera un ancien contenu éventuel).

À présent, étudiez le court programme qui suit:

```
boucle:
    MOV R0, #0xffff
    PUSH {R0}
    B boucle
```

#### Exercice 31

Faites tourner ce programme jusqu'à ce qu'il termine.

Décrire ce que le programme a fait et la raison de son arrêt.

Où s'est-il terminé c'est-à-dire quelle était la dernière position en mémoire qui a été modifiée? (indice: regarder le contenu de la mémoire de la page 000).
____________

Ce que vous avez dû découvrir incidemment c'est que l'ARMlite, comme la plupart des processeurs modernes, sait où se situe la fin de votre programme (fin incluant aussi toute zone de données déclarée dans le code assembleur). Il permettra à la pile de s'étendre dans toutes la mémoire disponible (dans notre cas, presque 1 megaoctet) mais pas d'écrire par dessus vos propres instructions ou les zones de données déclarées. 

## Sous-routines

Le code qui suit permet de dessiner n'importe quel pixel, en mode moyenne résolution, pourvu qu'on précise d'une part ses coordonnées X, Y dans les registres R0, R1 respectivement, d'autre part sa couleur dans le registre R3:

```
    MOV R0, #32   // X (0-63)
    MOV R1, #24   // Y (0-47)
    MOV R2, #.red // couleur

peindrePixel: MOV R3, #.PixelScreen
    LSL R4, R0, #2    // multiplier X par 4 ...
    LSL R5, R1, #8    // ... puis Y par 256
    ADD R5, R5, R4    // de façon à obtenir l'index du pixel (X,Y)
    STR R2, [R3+R5]
    HALT
```

Nous pourrions mettre ce code dans une boucle de façon à peindre, disons, une ligne, mais si nous voulions juste peindre deux pixels arbitraires, il nous faudrait dupliquer les 5 instructions après `peindrePixel` pour y parvenir. Cela contredirait le principe DRY (*Don't Repeat Yourself*) qui doit être suivi par tout bon programmeur. Nous pourrions facilement réaliser des branchement vers ce code depuis de nombreux endroits; mais comment pourrions-nous préciser, à la fin de cette routine, de revenir en arrière (*branch back*) à un *endroit du code différent à chaque fois*?  

La solution repose sur l'idée de **sous-routine**. Le code qui suit montre un code similaire à celui de `peindrePixel` transformé en sous-routine laquelle est invoquée deux fois:

```
    MOV R0, #32   // X (0-63)
    MOV R1, #24   // Y (0-47)
    MOV R2, #.red // couleur
    BL peindrePixel
    MOV R0, #37
    MOV R1, #19
    MOV R2, #.green
    BL peindrePixel
    HALT

// Sous-routine
peindrePixel: MOV R3, #.PixelScreen
    LSL R4, R0, #2    // multiplier X par 4 ...
    LSL R5, R1, #8    // ... puis Y par 256
    ADD R5, R5, R4    // de façon à obtenir l'index du pixel (X,Y)
    STR R2, [R3+R5]
    RET
```

*Notes*:
- Après avoir configurée les valeurs requises dans R0, R1 et R2, nous effectuons un branchement vers `peindrePixel` en utilisant cette fois-ci l'instruction `BL` (`Branch with Link back` - branchement avec retour) plutôt qu'un branchement `B` régulier.
- À la fin de la sous-routine `peindrePixel`, on trouve une nouvelle instruction `RET` (`Return`) laquelle ramène l'exécution *à l'instruction qui suit la dernière instruction `BL` invoquée*.
- Ici, on peut considérer les valeurs détenues par les registres R0, R1 et R2 comme des «paramètres» fournies à la sous-routine.

### Encart - The Wheeler Jump

à faire...

#### Exercice 32

Faites tourner ce programme et vérifier ses opérations. Étendez le avec un troisième appel de façon à peindre un autre pixel, *en modifiant seulement l'un des trois paramètres*. Prendre une capture d'écran afin de montrer votre code ainsi que cet autre pixel.

À présent, faites à nouveau tourner le programme, en utilisant le mode pas à pas. À chaque pas, observer attentivement les valeur des registres `PC` et `LR`, puis décrire, en détaillant, comment ces deux registres sont utilisés à la fois durant l'exécution normale et lorsque la sous-routine est invoquée.
________

Le registre `LR` est nommé ainsi pour `Link Register` (registre lien) - comme vous l'aviez peut être deviné. Il conserve le lien de retour produit par l'instruction «Branch with Link back».

L'un des principes qui président à toute bonne programmation est «la séparation de l'interface (moyen d'interragir avec) et de l'implémentation (réalisation sous jacente)»: nous devrions être capable d'invoquer une sous-routine simplement en connaissant son étiquette et les différents paramètres (s'il y en a!) dont elle a besoin. (Une sous-routine peut aussi agir comme une fonction en renvoyant un résultat, en le plaçant dans un ou plusieurs registres spécifiés). Nous ne devrions pas avoir à nous soucier (ou même simplement avoir une idée) de comment la sous-routine fonctionne en interne.

Mais il y a tout de même un problème potentiel... *Avant de lire la suite*, voyez si vous êtes en mesure de vous figurez de quel problème il peut s'agir. Essayer de vous figurer l'invocation de la routine `peindrePixel` depuis un grand nombre d'endroits dans un programme bien plus complexe et conséquent...

... Le problème est que la routine `peindrePixel` utilise les registres R3, R4 et R5, et que dans un plus grand programme nous pourrions avoir besoin de ces registres pour d'autres tâches. Utiliser la sous-routine sans inspecter son code interne pourrait détruire des données que nous aurions placés dans ces registres sans que nous nous en rendions compte, produisant ainsi des erreurs qui pourraient ne pas être remarquées initialement. Nous pourrions alors ré-écrire notre routine `peindrePixel` pour qu'elle utilise quelques autres registres mais le problème de base ne disparaît pas pour autant.

La solution à ce problème utilise la pile système:

```
// peint un pixel en mode med-res
// Préciser son abscisse X (0-63) en R0
// son ordonnée Y (0-47) en R1
// sa couleur en R2
peindrePixel:
    PUSH {R3-R5, LR}
    MOV R3, #.PixelScr
    LSL R4, R0, #2
    LSL R5, R1, #8
    ADD R5, R5, R4
    STR R2, [R3+R5]
    POP {R3-R5, LR}
    RET
```

*Notes*:
- Les commentaires ont été placés en avant du «point d'entrée» de la sous-routine. Ils expliquent son *interface*: tout ce que vous avez besoin de savoir pour l'utiliser (sans avoir besoin de consulter son code interne - son *implémentation*).
- La première instruction de `peindrePixel` empile - `PUSH` - les valeurs de plusieurs registres (R3, R4, R5 et LR) sur la pile système - en incluant toutes les valeurs que cette routine modifie ou pourrait modifier: on effectue ainsi une *sauvegarde* de ces valeurs.
- L'avant dernière instruction dépile les valeurs - `POP` - des registres précédemment sauvegardés dans la pile système: cela a pour effet de restaurer leurs valeurs dans l'état où elles se trouvaient avant que démarre la sous-routine.
- Les seuls registres non restaurés à leur valeur initiale seraient ceux qui nous servirait à fournir les valeurs produites par la routine en vue d'une réutilisation (valeurs de retour). Mais cette routine ne produit pas valeur en retour, elle se contente d'afficher un pixel à l'écran en écrivant en mémoire.
- Les instructions `PUSH` et `POP` peuvent sauvegarder/restaurer n'importe quel registre général R0 à R12 et/ou le registre LR, soit en les séparant par une virgule, ou en donnant un intervalle de registres numéroté, ou en combinant ces deux possibilités.
- La liste des registres pour `PUSH` et `POP` devrait être identique dans ce cas de figure: c'est alors la responsabilité du processeur de s'assurer que les valeurs reviennent bien dans les registres appropriés indépendamment du mécanisme interne - LIFO pour (*Last In Firt Out* - dernier entré, premier sorti) de la pile.

Mais pourquoi a-t-on inclut `LR` dans la liste des registres à sauvegarder/restaurer, alors que nous ne le modifions pas explicitement dans le code et que cet exemple fonctionnait déjà correctement?

La réponse est que, dans un programme plus large, nous aurons des sous-routines qui appellent d'autres sous-routines (nous pouvons même avoir des sous-routines qui s'appellent ... elle-même! - on dit *récursivement* - c'est un cas très commun - et c'est la pile qui rend cela possible). Cependant, le registre `LR` ne peut mémoriser qu'une seule adresse (lien) de retour à la fois. En sauvant et en restaurant ce registre `LR` à  l'intérieur de la sous-routine, on s'assure que l'adresse de retour ne sera pas écrasée lors d'un appel interne éventuel vers une autre sous-routine. Il est vrai que notre routine exemple ne contient pas d'instruction `BL`. Mais, *croyez-nous*, le fait de ne pas sauvegarder/restaurer ce registre lorsque vous en avez besoin peut conduire à des bugs très difficiles à diagnostiquer - ainsi *c'est une bonne pratique de toujours sauvegarder/restaurer le registre LR dans toutes les sous-routines, même si cela occasionne une petite surcharge*.

## Une sous-routine de multiplication

La sous-routine suivante multiplie deux entiers:

```
// Multiplie deux entier situé dans R0 et R1 et renvoie le produit dans R2
// Le produit doit tenir dans 32 bits pour être correct.
mult:
    PUSH {R0,R1,R3,LR}
    MOV R2, #0
deDroiteAGauche:
    AND R3, R1, #1
    CMP R3, #0
    BEQ sauter
    ADD R2, R2, R0
sauter:
    LSR R1, R1, #1
    CMP R1, #0
    BEQ fin
    LSL R0, R0, #1
    B deDroiteAGauche
fin:
    POP {R0,R1,R3,LR}
    RET
```

#### Exercice 33

Écrire un programme qui appelle cette sous-routine dans une boucle pour afficher une table de multiplication pour chaque *chiffre* décimal à partir de 1. Par exemple, si l'utilisateur saisi 5, la console devrait affichée:

    1 x 5 = 5
    2 x 5 = 10
    ...
    9 x 5 = 45

Donner votre programme et prendre une capture d'écran de la table de 7 (autant que la console permet de voir).
____